In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-3/pytorch/8b-chat/1/consolidated.00.pth
/kaggle/input/llama-3/pytorch/8b-chat/1/LICENSE
/kaggle/input/llama-3/pytorch/8b-chat/1/MODEL_CARD.md
/kaggle/input/llama-3/pytorch/8b-chat/1/params.json
/kaggle/input/llama-3/pytorch/8b-chat/1/model.py
/kaggle/input/llama-3/pytorch/8b-chat/1/USE_POLICY.md
/kaggle/input/llama-3/pytorch/8b-chat/1/Llama3_Repo.jpeg
/kaggle/input/llama-3/pytorch/8b-chat/1/example_text_completion.py
/kaggle/input/llama-3/pytorch/8b-chat/1/test_tokenizer.py
/kaggle/input/llama-3/pytorch/8b-chat/1/requirements.txt
/kaggle/input/llama-3/pytorch/8b-chat/1/tokenizer.py
/kaggle/input/llama-3/pytorch/8b-chat/1/eval_details.md
/kaggle/input/llama-3/pytorch/8b-chat/1/generation.py
/kaggle/input/llama-3/pytorch/8b-chat/1/__init__.py
/kaggle/input/llama-3/pytorch/8b-chat/1/example_chat_completion.py
/kaggle/input/llama-3/pytorch/8b-chat/1/tokenizer.model
/kaggle/input/llama-3/pytorch/8b-chat/1/setup.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model.saf

In [61]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, multilabel_confusion_matrix

# Define the labels
labels = ['vasps', 'conduct', 'investor', 'person', 'code of conduct', 'transactions', 'audit', 'pep', 
          'sanctions', 'operator', 'anti-bribery', 'transaction', 'due diligence', 'reporting', 
          'regulations', 'travel rule', 'risk assessment', 'virtual assets', 'identity', 'insider dealing', 
          'monitoring', 'rules', 'compliance', 'requirements', 'oversight', 'legislation', 'legal', 'aml', 
          'circular', 'disclosure', 'criminal', 'record keeping', 'controls', 'entities', 'client', 'vasp', 
          'policy', 'cdd', 'kyc', 'cft', 'risk management', 'dlt', 'screening', 'guideline', 'license', 
          'regulator', 'policies', 'investor protection', 'safeguard', 'suspicious transactions', 'risks', 
          'transfer', 'rulebook', 'authority', 'beneficiary information']

#labels = ['vasps', 'conduct', 'investor']

labels_str = ", ".join(labels)





In [62]:
len(labels)

55

In [3]:
def generate_prompt(data_point):
    assigned_labels = [label for label in labels if data_point[label] == 1]
    assigned_labels_str = ", ".join(assigned_labels)
    return f"""
    Identify the most relevant regulatory topics from the list: '''{labels_str}'''. 
    Use only the listed topics and return up to 5 as a comma-separated list. Leave blank if none apply.

    Text: [{data_point["Content"]}]
    Labels: {assigned_labels}
    """.strip()


def generate_test_prompt(data_point):
    return f"""
    Identify the most relevant regulatory topics from the list: '''{labels_str}'''. 
    Use only the listed topics and return up to 5 as a comma-separated list. Leave blank if none apply.

    Text: [{data_point["Content"]}]
    Labels: 
    """.strip()


In [4]:
df = pd.read_csv("/kaggle/input/crypto-regulations-yirifi/preprocessed_one_hot_encode.csv")
df = df.drop("Unnamed: 0", axis=1)
df = df.sample(frac=0.01, random_state=42)


In [5]:
df.head()

,Content,client,guideline,rules,travel rule,regulator,criminal,authority,pep,risk management,...,conduct,monitoring,sanctions,legal,requirements,operator,safeguard,anti-bribery,suspicious transactions,record keeping
3999,2. vasps shall ensure that all members of the ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1718,"b. experience in the virtual assets sector, in...",0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1907,3. prior to agreeing a sale to any client or i...,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3129,(d) obtaining a similar or comparable document...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
248,content dwtca means the dubai world trad...,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Simulate train, eval, and test splits
X_train = df.sample(frac=0.7, random_state=42)  # 70% for training
X_temp = df.drop(X_train.index)
X_eval = X_temp.sample(frac=0.5, random_state=42)  # 15% for evaluation
X_test = X_temp.drop(X_eval.index)  # 15% for testing

In [7]:
print(X_train.shape, X_eval.shape, X_test.shape)

(38, 56) (8, 56) (8, 56)


In [8]:
df.columns

Index(['Content', 'client', 'guideline', 'rules', 'travel rule', 'regulator',
       'criminal', 'authority', 'pep', 'risk management', 'policy', 'identity',
       'virtual assets', 'kyc', 'transfer', 'risk assessment', 'investor',
       'vasp', 'compliance', 'beneficiary information', 'screening',
       'regulations', 'disclosure', 'vasps', 'legislation', 'due diligence',
       'license', 'controls', 'rulebook', 'entities', 'transaction', 'cft',
       'policies', 'oversight', 'reporting', 'aml', 'risks',
       'investor protection', 'insider dealing', 'person', 'cdd', 'dlt',
       'circular', 'transactions', 'code of conduct', 'audit', 'conduct',
       'monitoring', 'sanctions', 'legal', 'requirements', 'operator',
       'safeguard', 'anti-bribery', 'suspicious transactions',
       'record keeping'],
      dtype='object')

In [9]:
#labels = list(df.columns)[1:]

In [10]:
labels

['vasps',
 'conduct',
 'investor',
 'person',
 'code of conduct',
 'transactions',
 'audit',
 'pep',
 'sanctions',
 'operator',
 'anti-bribery',
 'transaction',
 'due diligence',
 'reporting',
 'regulations',
 'travel rule',
 'risk assessment',
 'virtual assets',
 'identity',
 'insider dealing',
 'monitoring',
 'rules',
 'compliance',
 'requirements',
 'oversight',
 'legislation',
 'legal',
 'aml',
 'circular',
 'disclosure',
 'criminal',
 'record keeping',
 'controls',
 'entities',
 'client',
 'vasp',
 'policy',
 'cdd',
 'kyc',
 'cft',
 'risk management',
 'dlt',
 'screening',
 'guideline',
 'license',
 'regulator',
 'policies',
 'investor protection',
 'safeguard',
 'suspicious transactions',
 'risks',
 'transfer',
 'rulebook',
 'authority',
 'beneficiary information']

In [11]:

# Generate prompts for training and evaluation
X_train = pd.DataFrame(X_train.apply(generate_prompt, axis=1), columns=['Content'])
X_eval = pd.DataFrame(X_eval.apply(generate_prompt, axis=1), columns=['Content'])

# Extract the true labels (y_true) for testing
y_true = X_test[labels].values




In [12]:
X_train['Content']

2394    Identify the most relevant regulatory topics f...
1554    Identify the most relevant regulatory topics f...
4828    Identify the most relevant regulatory topics f...
626     Identify the most relevant regulatory topics f...
5300    Identify the most relevant regulatory topics f...
2932    Identify the most relevant regulatory topics f...
994     Identify the most relevant regulatory topics f...
2024    Identify the most relevant regulatory topics f...
3129    Identify the most relevant regulatory topics f...
4587    Identify the most relevant regulatory topics f...
3296    Identify the most relevant regulatory topics f...
3626    Identify the most relevant regulatory topics f...
4786    Identify the most relevant regulatory topics f...
2521    Identify the most relevant regulatory topics f...
371     Identify the most relevant regulatory topics f...
248     Identify the most relevant regulatory topics f...
1074    Identify the most relevant regulatory topics f...
4290    Identi

In [13]:
X_test.head(3)

,Content,client,guideline,rules,travel rule,regulator,criminal,authority,pep,risk management,...,conduct,monitoring,sanctions,legal,requirements,operator,safeguard,anti-bribery,suspicious transactions,record keeping
3928,a. vasps must have procedures for ensuring all...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3064,(b) products or services distributed or sold d...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2095,(iv) transparency of beneficial ownership[fn4],0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Generate test prompts
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=['Content'])

In [15]:
X_test['Content']

3928    Identify the most relevant regulatory topics f...
3064    Identify the most relevant regulatory topics f...
2095    Identify the most relevant regulatory topics f...
3752    Identify the most relevant regulatory topics f...
240     Identify the most relevant regulatory topics f...
765     Identify the most relevant regulatory topics f...
2370    Identify the most relevant regulatory topics f...
84      Identify the most relevant regulatory topics f...
Name: Content, dtype: object

In [43]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, multilabel_confusion_matrix

# Simulate model predictions
# For demonstration purposes, let's assume y_pred is the same as y_true
# In a real scenario, y_pred would be generated by your model
# Simulated model output (as text)
model_output = [
    'regulator, legal, authority',
    'authority',
    'legal'

]

# Function to convert text predictions to binary matrix
def parse_predictions(predictions, labels):
    y_pred = np.zeros((len(predictions), len(labels)), dtype=int)
    for i, pred in enumerate(predictions):
        pred_labels = pred.split(', ')
        for label in pred_labels:
            if label in labels:
                y_pred[i, labels.index(label)] = 1
    return y_pred

# Parse model output to binary matrix
y_pred = parse_predictions(model_output, labels)


    

In [47]:
import numpy as np

# Function to convert text predictions to binary matrix
def parse_predictions(predictions, labels):
    y_pred = np.zeros((len(predictions), len(labels)), dtype=int)
    for i, pred_list in enumerate(predictions):
        for label in pred_list:
            if label in labels:
                y_pred[i, labels.index(label)] = 1
    return y_pred

# Example model output and labels
model_output = [
    ['vasps', 'transactions', 'client', 'withdrawals', 'authorised'],
    ['vasps', 'conduct', 'investor', 'person', 'code of conduct', 'transactions', 'audit', 'pep', 'sanctions', 'operator'],
    ['vasps', 'conduct', 'investor', 'person', 'code of conduct', 'transactions', 'audit', 'pep', 'sanctions', 'operator'],
    ['vasps', 'audit', 'report', 'regulations', 'requirements', 'rules', 'compliance', 'oversight', 'regulator', 'policy'],
    ['vasps', 'conduct', 'investor', 'person', 'code of conduct', 'transactions', 'audit', 'pep', 'sanctions', 'operator'],
    ['vasps', 'conduct', 'investor', 'person', 'code of conduct', 'transactions', 'audit', 'pep', 'sanctions', 'operator'],
    ['source of wealth', 'individual', 'body of wealth', 'total assets', 'origin', 'size', 'picture', 'acquired', 'fi', 'assets'],
    ['varas', 'exempt entity', 'no objection confirmation', 'regulation iv.a.6.a', 'absolute discretion']
]

#labels = ['vasps', 'conduct', 'investor', 'person', 'code of conduct', 'transactions', 'audit', 'pep', 'sanctions', 'operator', 'anti-bribery', 'transaction', 'due diligence', 'reporting', 'regulations', 'travel rule', 'risk assessment', 'virtual assets', 'identity', 'insider dealing', 'monitoring', 'rules', 'compliance', 'requirements', 'oversight', 'legislation', 'legal', 'aml', 'circular', 'disclosure', 'criminal', 'record keeping', 'controls', 'entities', 'client', 'vasp', 'policy', 'cdd', 'kyc', 'cft', 'risk management', 'dlt', 'screening', 'guideline', 'license', 'regulator', 'policies', 'investor protection', 'safeguard', 'suspicious transactions', 'risks', 'transfer', 'rulebook', 'authority', 'beneficiary information', 'source of wealth', 'individual', 'body of wealth', 'total assets', 'origin', 'size', 'picture', 'acquired', 'fi', 'assets', 'varas', 'exempt entity', 'no objection confirmation', 'regulation iv.a.6.a', 'absolute discretion']

# Parse model output to binary matrix
y_pred = parse_predictions(model_output, labels)

y_pred


array([[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1,

In [44]:
#y_pred = y_true

In [45]:
y_pred

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [19]:
y_true[3]

array([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [20]:
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, f1_score

# Evaluation function
def evaluate(y_true, y_pred, labels):
    # Calculate subset accuracy (exact match ratio)
    accuracy = accuracy_score(y_true, y_pred)
    print(f'Subset Accuracy: {accuracy:.3f}')
    
    # Calculate F1 score
    f1_micro = f1_score(y_true, y_pred, average='micro')
    print(f'F1 Score (Micro): {f1_micro:.3f}')
    
    # Calculate AUC-ROC
    auc_roc = roc_auc_score(y_true, y_pred, average='micro')
    print(f'AUC-ROC: {auc_roc:.3f}')
    
    # Generate classification report for multilabel
#     class_report = classification_report(y_true, y_pred, target_names=labels)
#     print('\nClassification Report:')
#     print(class_report)
    
    

# Evaluate the model
evaluate(y_true, y_pred, labels)


Subset Accuracy: 1.000
F1 Score (Micro): 1.000
AUC-ROC: 1.000


# Start Llama3 finetuning 

In [21]:
!pip install -q -U torch --index-url https://download.pytorch.org/whl/cu117
!pip install -q -U -i https://pypi.org/simple/ bitsandbytes
!pip install -q -U transformers=="4.40.0"
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl
!pip install -q -U peft
!pip install -q -U tensorboard

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.
tensorflow 2.15.0 requires tensorboard<2.16,>=2.15, but you have tensorboard 2.17.0 which is incompatible.


In [22]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [23]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

2024-06-12 11:32:21.144168: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 11:32:21.144312: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 11:32:21.241774: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"working on {device}")

working on cuda:0


In [25]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [26]:
train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)

In [27]:
train_data

Dataset({
    features: ['Content', '__index_level_0__'],
    num_rows: 38
})

In [28]:
train_data['Content'][0]

"Identify the most relevant regulatory topics from the list: '''vasps, conduct, investor, person, code of conduct, transactions, audit, pep, sanctions, operator, anti-bribery, transaction, due diligence, reporting, regulations, travel rule, risk assessment, virtual assets, identity, insider dealing, monitoring, rules, compliance, requirements, oversight, legislation, legal, aml, circular, disclosure, criminal, record keeping, controls, entities, client, vasp, policy, cdd, kyc, cft, risk management, dlt, screening, guideline, license, regulator, policies, investor protection, safeguard, suspicious transactions, risks, transfer, rulebook, authority, beneficiary information'''. \n    Use only the listed topics and return up to 5 as a comma-separated list. Leave blank if none apply.\n\n    Text: [when continuing an existing business\r\nrelationship with a customer who is or whose\r\nbeneficial owner is a hong kong pep or an\r\ninternational organisation pep where the\r\nrelationship subseq

In [29]:
eval_data

Dataset({
    features: ['Content', '__index_level_0__'],
    num_rows: 8
})

In [75]:
model_name = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype=compute_dtype,
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

max_seq_length = 1024
tokenizer = AutoTokenizer.from_pretrained(model_name, max_seq_length=max_seq_length)
tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [76]:
import re

def extract_labels(text):
    # Define the regular expression pattern to extract the list within the 'Labels:' section
    pattern = r"Labels:\s*\[(.*?)(\]|$)"
    match = re.search(pattern, text)
    
    if match:
        labels_str = match.group(1)  # Get the string within the brackets
        # Split the string by commas and strip any extra whitespace or quotes
        labels = [label.strip().strip("'").strip('"') for label in labels_str.split(',')]
        return labels[:10]  # Return only the first 5 labels if there are more
    else:
        return []

# Example usage
# text = "Text: [maximum stored value exceeding hk 8,000;]\nLabels: [vasps, conduct, investor, person, code of conduct, transactions, audit, pep, sanctions, operator, anti-bribery, transaction, due diligence, reporting, regulations, travel rule, risk assessment, virtual assets, identity, insider"
# labels = extract_labels(text)
# print(labels)


In [83]:
from transformers import pipeline
from tqdm import tqdm
import numpy as np

def predict(test, model, tokenizer, labels):
    y_pred = []
    for i in tqdm(range(len(test))):
        prompt = test.iloc[i]["Content"]
        #print(prompt)
        pipe = pipeline(task="text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens=30, 
                        temperature=0.5)
        result = pipe(prompt)
        print(result[0]['generated_text'])
        print()
        #answer = result[0]['generated_text'].split("=")[-1].strip()
        answer = extract_labels(result[0]['generated_text'])
        print(answer)
        
        #print(answer)
        y_pred.append(answer)
    
    # Convert predictions to binary matrix
    print(y_pred)
    
    def parse_predictions(predictions, labels):
        y_pred = np.zeros((len(predictions), len(labels)), dtype=int)
        for i, pred_list in enumerate(predictions):
            for label in pred_list:
                if label in labels:
                    y_pred[i, labels.index(label)] = 1
        return y_pred

    y_pred_binary = parse_predictions(y_pred, labels)
    return y_pred_binary






In [84]:
len(labels)

55

In [85]:
labels = labels # All possible labels

y_pred = predict(X_test, model, tokenizer, labels)

  0%|          | 0/8 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
 12%|█▎        | 1/8 [00:02<00:18,  2.65s/it]

Identify the most relevant regulatory topics from the list: '''vasps, conduct, investor, person, code of conduct, transactions, audit, pep, sanctions, operator, anti-bribery, transaction, due diligence, reporting, regulations, travel rule, risk assessment, virtual assets, identity, insider dealing, monitoring, rules, compliance, requirements, oversight, legislation, legal, aml, circular, disclosure, criminal, record keeping, controls, entities, client, vasp, policy, cdd, kyc, cft, risk management, dlt, screening, guideline, license, regulator, policies, investor protection, safeguard, suspicious transactions, risks, transfer, rulebook, authority, beneficiary information'''. 
    Use only the listed topics and return up to 5 as a comma-separated list. Leave blank if none apply.

    Text: [a. vasps must have procedures for ensuring all withdrawals from a client account are authorised]
    Labels: ['vasps', 'transactions', 'client', 'withdrawals', 'authorised']
    """

    relevant_topi

 25%|██▌       | 2/8 [00:05<00:15,  2.63s/it]

Identify the most relevant regulatory topics from the list: '''vasps, conduct, investor, person, code of conduct, transactions, audit, pep, sanctions, operator, anti-bribery, transaction, due diligence, reporting, regulations, travel rule, risk assessment, virtual assets, identity, insider dealing, monitoring, rules, compliance, requirements, oversight, legislation, legal, aml, circular, disclosure, criminal, record keeping, controls, entities, client, vasp, policy, cdd, kyc, cft, risk management, dlt, screening, guideline, license, regulator, policies, investor protection, safeguard, suspicious transactions, risks, transfer, rulebook, authority, beneficiary information'''. 
    Use only the listed topics and return up to 5 as a comma-separated list. Leave blank if none apply.

    Text: [(b) products or services distributed or sold directly to the
customer.]
    Labels: ['vasps', 'conduct', 'investor', 'person', 'code of conduct', 'transactions', 'audit', 'pep', '

['vasps', 'conduct'

 38%|███▊      | 3/8 [00:07<00:13,  2.62s/it]

...

Identify the most relevant regulatory topics from the list: '''vasps, conduct, investor, person, code of conduct, transactions, audit, pep, sanctions, operator, anti-bribery, transaction, due diligence, reporting, regulations, travel rule, risk assessment, virtual assets, identity, insider dealing, monitoring, rules, compliance, requirements, oversight, legislation, legal, aml, circular, disclosure, criminal, record keeping, controls, entities, client, vasp, policy, cdd, kyc, cft, risk management, dlt, screening, guideline, license, regulator, policies, investor protection, safeguard, suspicious transactions, risks, transfer, rulebook, authority, beneficiary information'''. 
    Use only the listed topics and return up to 5 as a comma-separated list. Leave blank if none apply.

    Text: [maximum stored value exceeding hk 8,000;]
    Labels: [vasps, conduct, investor, person, code of conduct, transactions, audit, pep, sanctions, operator, anti-bribery, transaction

['vasps', 'conduct',

 88%|████████▊ | 7/8 [00:18<00:02,  2.67s/it]

Identify the most relevant regulatory topics from the list: '''vasps, conduct, investor, person, code of conduct, transactions, audit, pep, sanctions, operator, anti-bribery, transaction, due diligence, reporting, regulations, travel rule, risk assessment, virtual assets, identity, insider dealing, monitoring, rules, compliance, requirements, oversight, legislation, legal, aml, circular, disclosure, criminal, record keeping, controls, entities, client, vasp, policy, cdd, kyc, cft, risk management, dlt, screening, guideline, license, regulator, policies, investor protection, safeguard, suspicious transactions, risks, transfer, rulebook, authority, beneficiary information'''. 
    Use only the listed topics and return up to 5 as a comma-separated list. Leave blank if none apply.

    Text: [source of wealth refers to the origin of an
individual   s entire body of wealth (i.e. total assets).
this information will usually give an indication as to
the size of wealth the customer would be ex

100%|██████████| 8/8 [00:21<00:00,  2.65s/it]

Identify the most relevant regulatory topics from the list: '''vasps, conduct, investor, person, code of conduct, transactions, audit, pep, sanctions, operator, anti-bribery, transaction, due diligence, reporting, regulations, travel rule, risk assessment, virtual assets, identity, insider dealing, monitoring, rules, compliance, requirements, oversight, legislation, legal, aml, circular, disclosure, criminal, record keeping, controls, entities, client, vasp, policy, cdd, kyc, cft, risk management, dlt, screening, guideline, license, regulator, policies, investor protection, safeguard, suspicious transactions, risks, transfer, rulebook, authority, beneficiary information'''. 
    Use only the listed topics and return up to 5 as a comma-separated list. Leave blank if none apply.

    Text: [b. vara has sole and absolute discretion at any time in deciding whether an entity is an
exempt entity and/or granting any no objection confirmation under regulation iv.a.6.a
above.]
    Labels: ['var

In [86]:
y_true.shape

(8, 55)

In [87]:
y_pred.shape

(8, 55)

In [88]:
evaluate(y_true, y_pred, labels)

Subset Accuracy: 0.000
F1 Score (Micro): 0.125
AUC-ROC: 0.552


In [68]:
evaluate(y_true, y_pred, labels)

Subset Accuracy: 0.000
F1 Score (Micro): 0.125
AUC-ROC: 0.552
